# Separate fics into done, 100-fic chunks for parallelizing pipeline

In [2]:
import os

data_dirpath = '/usr0/home/mamille2/erebor/fanfiction-project/data/ao3/harrypotter/emnlp_dataset_6k/' # misty

all_fic_ids = [int(fname[:-4]) for fname in os.listdir(os.path.join(data_dirpath, 'fics'))]
len(all_fic_ids)

5594

# Get fic_ids to run the pipeline

In [1]:
# Load existing data (5+ paragraphs with at least one pairing together)

import pandas as pd

# Load data
data = pd.read_pickle('/usr2/mamille2/fanfiction-project/data/features/relationship_prediction.pkl')
len(data)
print(data.columns)

pairings = data['pairing'].unique()
pairings

Index(['fic_id', 'pairing', 'pairing_embedding_embs',
       'pairing_embedding_unigrams', 'relationship', 'selected_relationships',
       'is_romantic'],
      dtype='object')


array([('draco', 'harry'), ('hermione', 'ron'), ('ginny', 'harry'),
       ('draco', 'hermione'), ('harry', 'hermione'), ('harry', 'ron')], dtype=object)

In [2]:
canon_relationship_map = {
        ('draco', 'harry'): False,
        ('hermione', 'ron'): True,
        ('ginny', 'harry'): True,
        ('draco', 'hermione'): False,
        ('harry', 'hermione'): False,
        ('harry', 'ron'): False,
                         }

relationship_type_map = {
        ('draco', 'harry'): True,
        ('hermione', 'ron'): False,
        ('ginny', 'harry'): False,
        ('draco', 'hermione'): False,
        ('harry', 'hermione'): False,
        ('harry', 'ron'): True,
                         }

data['is_canon'] = [(is_romantic and canon_relationship_map[pairing]) or (not is_romantic and not canon_relationship_map[pairing]) \
                    for is_romantic, pairing in zip(data['is_romantic'], data['pairing'])]

data['is_mm'] = data['pairing'].map(lambda x: relationship_type_map[x])

In [3]:
# Save out
data.to_pickle('/usr2/mamille2/fanfiction-project/data/features/relationship_prediction.pkl')

In [4]:
fic_ids = sorted(data['fic_id'].unique())
len(fic_ids)

44647

In [ ]:
# Copy those fic_ids to new directory for pipeline
import os
from tqdm import tqdm_notebook as tqdm

data_dirpath = '/usr2/scratch/fanfic/ao3_harrypotter_text/stories/'

for fic_id in tqdm(fic_ids):
    fic_fpath = 